# PDF Extraction

In [2]:
!pip install pypdf

In [3]:
s3_bucket = 'storage-for-korean-chatbot-with-rag-677146750822-us-west-2'
# key = 'pdf/sample.pdf'
key = 'pdf/jw.pdf'

In [4]:
import boto3
from pypdf import PdfReader      
from io import BytesIO

s3r = boto3.resource("s3")
doc = s3r.Object(s3_bucket, key)

Byte_contents = doc.get()['Body'].read()

reader = PdfReader(BytesIO(Byte_contents))

## Text 

In [5]:
texts = []

for i, page in enumerate(reader.pages):
    texts.append(page.extract_text())
    
    contents = '\n'.join(texts)    

In [6]:
contents[:500]

'* 기간: \n년\n월\n일\n 년\n월\n일\n이그제큐티브 라운지이용인원추가시\n성인\n인\n만원\n어린이\n 개월\n만\n세\n인\n만원\n예약마감날짜\n월\n 일\n월\n 일\n월\n일\n* 문의: 객실예약과\n지화영지배인\n2021년기업체임직원분들을 위한특별한휴양프로모션으로\nJW 메리어트 호텔서울에서 차원이다른럭셔리를 즐겨보시기 바랍니다 .\n객실타입이그제큐티브 디럭스\n(이그제큐티브 라운지\n 인포함)\n일요일 -금요일\n 원\n토요일\n 원\n객실타입그리핀스위트\n(이그제큐티브 라운지\n 인포함)\n일요일 -금요일\n 원\n토요일\n 원\n혜택\n 혜택\n 혜택\n이그제큐티브 라운지\n인이용가능\n조식\n이브닝오르되브르\n이브닝칵테일\n디저트\n당첨경품이벤트\n쿠폰증정\n(호텔1박숙박권및레스토랑 & 바이용권 , \n호텔PB 상품, 화장품등) \n*투숙당 1매제공마르퀴스 피트니스 클럽 ,\n실내수영장및사우나\n무료이용\n•상기금액외\n세금과\n 의봉사료가 별도로부과됩니다\n•코로나\n 감염예방과확산방지를위한정부지침에따라\n호텔운영사항은변경될수있습니다\n메리어트 본보이가입시\n세이하자녀\n'

## Image

In [7]:
s3_prefix = 'pdf'
s3_client = boto3.client('s3')  
def extract_images_from_pdf(reader, key):
    picture_count = 1
    
    extracted_image_files = []
    for i, page in enumerate(reader.pages):        
        for image_file_object in page.images:
            img_name = image_file_object.name
            if img_name in extracted_image_files:
                # print('skip....')
                continue
            
            extracted_image_files.append(img_name)
            
            ext = img_name.split('.')[-1]            
            contentType = ""
            if ext == 'png':
                contentType = 'image/png'
            elif ext == 'jpg' or ext == 'jpeg':
                contentType = 'image/jpeg'
            elif ext == 'gif':
                contentType = 'image/gif'
            elif ext == 'bmp':
                contentType = 'image/bmp'
            elif ext == 'tiff' or ext == 'tif':
                contentType = 'image/tiff'
            elif ext == 'svg':
                contentType = 'image/svg+xml'
            elif ext == 'webp':
                contentType = 'image/webp'
            elif ext == 'ico':
                contentType = 'image/x-icon'
            elif ext == 'eps':
                contentType = 'image/eps'
            
            if contentType:                
                image_bytes = image_file_object.data

                pixels = BytesIO(image_bytes)
                pixels.seek(0, 0)
                            
                # get path from key
                objectName = (key[key.find(s3_prefix)+len(s3_prefix)+1:len(key)])
                folder = s3_prefix+'/files/'+objectName+'/'
                            
                img_key = folder+img_name                
                response = s3_client.put_object(
                    Bucket=s3_bucket,
                    Key=img_key,
                    ContentType=contentType,
                    Body=pixels
                )
                            
                # metadata
                img_meta = {   
                    'bucket': s3_bucket,
                    'key': img_key,
                    # 'url': path+img_key,
                    'ext': 'png',
                    'page': i+1,
                    'original': key
                }                            
                picture_count += 1                    
                extracted_image_files.append(img_key)

    return extracted_image_files

In [8]:
files = []

image_files = extract_images_from_pdf(reader, key)
for img in image_files:
    files.append(img)

image and mask size not matching: {'/Type': '/XObject', '/Subtype': '/Image', '/Width': 2, '/Height': 2, '/ColorSpace': ['/Indexed', '/DeviceRGB', 1, 'fa]ÿÿÿ'], '/BitsPerComponent': 1, '/Interpolate': False, '/SMask': IndirectObject(16, 0, 140428976776448)}
image and mask size not matching: {'/Type': '/XObject', '/Subtype': '/Image', '/Width': 2, '/Height': 2, '/ColorSpace': ['/Indexed', '/DeviceRGB', 1, 'fa]ÿÿÿ'], '/BitsPerComponent': 1, '/Interpolate': False, '/SMask': IndirectObject(23, 0, 140428976776448)}
image and mask size not matching: {'/Type': '/XObject', '/Subtype': '/Image', '/Width': 2, '/Height': 2, '/ColorSpace': ['/Indexed', '/DeviceRGB', 1, 'fa]ÿÿÿ'], '/BitsPerComponent': 1, '/Interpolate': False, '/SMask': IndirectObject(25, 0, 140428976776448)}
image and mask size not matching: {'/Type': '/XObject', '/Subtype': '/Image', '/Width': 2, '/Height': 2, '/ColorSpace': ['/Indexed', '/DeviceRGB', 1, 'fa]ÿÿÿ'], '/BitsPerComponent': 1, '/Interpolate': False, '/SMask': Indirect

### Page 단위로 이미지 저장하기

In [9]:
nImages = []
imgList = []
for i, page in enumerate(reader.pages):
    nImage = 0
    if '/XObject' in page['/Resources']:
        for j, image in enumerate(page['/Resources']['/XObject']):
            print(f"image[{j}]: {image}")                                 
            if image in imgList:
                continue    
            else:
                imgList.append(image)
                                                    
            Im = page['/Resources']['/XObject'][image]
            nImage = nImage+1
                            
    print(f"# of images of page[{i}] = {nImage}")
    nImages.append(nImage)

image[0]: /Image6
image[1]: /Image15
image[2]: /Image22
image[3]: /Image24
image[4]: /Image26
image[5]: /Image28
image[6]: /Image30
image[7]: /Image32
image[8]: /Image34
image[9]: /Image36
image[10]: /Image38
image[11]: /Image40
image[12]: /Image42
image[13]: /Image44
image[14]: /Image46
image[15]: /Image48
image[16]: /Image50
image[17]: /Image52
image[18]: /Image54
image[19]: /Image56
image[20]: /Image58
image[21]: /Image60
image[22]: /Image62
image[23]: /Image64
image[24]: /Image66
image[25]: /Image68
image[26]: /Image70
image[27]: /Image72
image[28]: /Image75
image[29]: /Image77
image[30]: /Image80
image[31]: /Image82
image[32]: /Image84
image[33]: /Image86
image[34]: /Image88
image[35]: /Image90
image[36]: /Image92
image[37]: /Image96
image[38]: /Image98
image[39]: /Image100
image[40]: /Image102
image[41]: /Image104
image[42]: /Image106
image[43]: /Image108
image[44]: /Image110
image[45]: /Image112
image[46]: /Image114
image[47]: /Image116
image[48]: /Image118
image[49]: /Image120


In [10]:
from PIL import Image

In [11]:
import fitz

pages = fitz.open(stream=Byte_contents, filetype='pdf')      
            
for i, page in enumerate(pages):
    imgInfo = page.get_image_info()
    width = height = 0
    for j, info in enumerate(imgInfo):
        bbox = info['bbox']
        print(f"page[{i}] -> bbox[{j}]: {bbox}")
        if (bbox[2]-bbox[0]>width or bbox[3]-bbox[1]>height) and (bbox[2]-bbox[0]<940 and bbox[3]-bbox[1]<520):
            width = bbox[2]-bbox[0]
            height = bbox[3]-bbox[1]
                        
    if nImages[i]>=4 or \
        (nImages[i]>=1 and (width==0 and height==0)) or \
        (nImages[i]>=1 and (width>=100 or height>=100)):
        # save current pdf page to image 
        pixmap = page.get_pixmap(dpi=200)  # dpi=300
        #pixels = pixmap.tobytes() # output: jpg
                        
        # convert to png
        img = Image.frombytes("RGB", [pixmap.width, pixmap.height], pixmap.samples)
        pixels = BytesIO()
        img.save(pixels, format='PNG')
        pixels.seek(0, 0)

        # get path from key
        objectName = (key[key.find(s3_prefix)+len(s3_prefix)+1:len(key)])
        folder = s3_prefix+'/captures/'+objectName+'/'
                                
        fname = 'img_'+key.split('/')[-1].split('.')[0]+f"_{i}"

        response = s3_client.put_object(
        Bucket=s3_bucket,
            Key=folder+fname+'.png',
            ContentType='image/png',
            Metadata = {
                "ext": 'png',
                "page": str(i)
            },
            Body=pixels
        )
                                                        
        files.append(folder+fname+'.png')   

page[0] -> bbox[0]: (48.84000015258789, 259.9200134277344, 196.3199920654297, 358.20001220703125)
page[0] -> bbox[1]: (85.46399688720703, 556.0099487304688, 111.71399688720703, 569.0899658203125)
page[0] -> bbox[2]: (118.58000183105469, 556.0099487304688, 133.52000427246094, 569.0899658203125)
page[0] -> bbox[3]: (140.57000732421875, 556.0099487304688, 147.77000427246094, 569.0899658203125)
page[0] -> bbox[4]: (156.0500030517578, 556.0099487304688, 163.010009765625, 569.0899658203125)
page[0] -> bbox[5]: (161.57000732421875, 556.0099487304688, 187.82000732421875, 569.0899658203125)
page[0] -> bbox[6]: (194.69000244140625, 556.0099487304688, 208.3699951171875, 569.0899658203125)
page[0] -> bbox[7]: (215.80999755859375, 556.0099487304688, 229.6699981689453, 569.0899658203125)
page[0] -> bbox[8]: (79.94400024414062, 573.6500244140625, 86.18399810791016, 585.4100341796875)
page[0] -> bbox[9]: (235.8699951171875, 573.6500244140625, 241.989990234375, 585.4100341796875)
page[0] -> bbox[10]: (

## Table

In [12]:
tables = []
for i, page in enumerate(pages):
    tab = page.find_tables()
    if tab.tables:
        print(tab[0].extract())
        tables.append(tab[0])

[['혜택1', '혜택2', '혜택3'], [None, '당첨경품이벤트\n쿠폰증정\n(호텔1박숙박권및레스토랑& 바이용권,\n호텔PB 상품, 화장품등)\n*투숙당1매제공', None]]


### Convert to pandas DataFrame

In [13]:
for tab in tables:
    df = tab.to_pandas()
    print("Table:")
    print(df)

Table:
    혜택1                  혜택2   혜택3
0  None  제큐티브디럭스\n이그제큐티브 라운지  None


### Convert to MarkDown

In [14]:
len(tables)

1

In [15]:
for tab in tables:
    md = tab.to_markdown()
    print(md)

|혜택1|혜택2|혜택3|
|---|---|---|
||제큐티브디럭스 이그제큐티브 라운지||




In [20]:
table_md = []
for i, page in enumerate(pages):
    tab = page.find_tables()
    if tab.tables:
        table_md.append(tab[0].to_markdown())
table_md

['|혜택1|혜택2|혜택3|\n|---|---|---|\n||당첨경품이벤트 쿠폰증정 (호텔1박숙박권및레스토랑&amp; 바이용권, 호텔PB 상품, 화장품등) *투숙당1매제공||\n\n']

### Table Info

In [16]:
for i, tab in enumerate(tables):
    print(f"index: {i}")
    print(f"bounding box: {tab.bbox}")  # bounding box of the full table
    print(f"top-left cell: {tab.cells[0]}")  # top-left cell
    print(f"bottom-right cell: {tab.cells[-1]}")  # bottom-right cell
    print(f"row count: {tab.row_count}, column count: {tab.col_count}") # row and column counts

index: 0
bounding box: (48.38100051879883, 367.6492614746094, 491.12049865722656, 492.38800048828125)
top-left cell: (48.38100051879883, 367.6492614746094, 184.95033264160156, 387.9800109863281)
bottom-right cell: (354.98000081380206, 367.6492614746094, 491.12049865722656, 387.9800109863281)
row count: 2, column count: 3


In [17]:
from PIL import Image

def extract_table_image(page, index, bbox):
    pixmap_ori = page.get_pixmap()
    print(f"width: {pixmap_ori.width}, height: {pixmap_ori.height}")
        
    pixmap = page.get_pixmap(dpi=200)  # dpi=300
    #pixels = pixmap.tobytes() # output: jpg
    
    # convert to png
    img = Image.frombytes("RGB", [pixmap.width, pixmap.height], pixmap.samples)
    print(f"width: {pixmap.width}, height: {pixmap.height}")
    
    rate_width = pixmap.width / pixmap_ori.width
    rate_height = pixmap.height / pixmap_ori.height
    print(f"rate_width={rate_width}, rate_height={rate_height}")
    
    crop_img = img.crop((bbox[0]*rate_width, bbox[1]*rate_height, bbox[2]*rate_width, bbox[3]*rate_height))
    
    pixels = BytesIO()
    crop_img.save(pixels, format='PNG')
    pixels.seek(0, 0)

    # get path from key
    objectName = (key[key.find(s3_prefix)+len(s3_prefix)+1:len(key)])
    folder = s3_prefix+'/captures/'+objectName+'/'
                                
    fname = 'table_'+key.split('/')[-1].split('.')[0]+f"_{index}"

    response = s3_client.put_object(
    Bucket=s3_bucket,
        Key=folder+fname+'.png',
        ContentType='image/png',
        Metadata = {
            "ext": 'png',
            "page": str(index)
        },
        Body=pixels
    )
                                                        
    files.append(folder+fname+'.png')   

In [21]:
tables = []
for i, page in enumerate(pages):
    page_tables = page.find_tables()
    
    if page_tables.tables:
        tab = page_tables[0]
        
        print(tab.to_markdown())    
        print(f"index: {i}")
        print(f"bounding box: {tab.bbox}")  # bounding box of the full table
        print(f"top-left cell: {tab.cells[0]}")  # top-left cell
        print(f"bottom-right cell: {tab.cells[-1]}")  # bottom-right cell
        print(f"row count: {tab.row_count}, column count: {tab.col_count}") # row and column counts
        print("\n\n")
        
        extract_table_image(page, i, tab.bbox)

|혜택1|혜택2|혜택3|
|---|---|---|
||당첨경품이벤트 쿠폰증정 (호텔1박숙박권및레스토랑&amp; 바이용권, 호텔PB 상품, 화장품등) *투숙당1매제공||


index: 0
bounding box: (48.38100051879883, 367.6492614746094, 491.12049865722656, 492.38800048828125)
top-left cell: (48.38100051879883, 367.6492614746094, 184.95033264160156, 387.9800109863281)
bottom-right cell: (354.98000081380206, 367.6492614746094, 491.12049865722656, 387.9800109863281)
row count: 2, column count: 3



width: 540, height: 792
width: 1500, height: 2200
rate_width=2.7777777777777777, rate_height=2.7777777777777777
